In [ ]:

# Save figures in specific place

knitr::opts_chunk$set(autodep        = TRUE,
                      cache          = FALSE,
                      cache.comments = TRUE,

                      # Save figures as pdf ?
                      dev = c("png", "pdf"),

                      # Include code?
                      echo           = TRUE,
                      warning = FALSE,

                      error          = FALSE,
                      fig.align      = "center",

                      # Path where figures are going to be store pdf single
                      # figures
                      fig.path       = paste0("./figures", "/"),
                      fig.width      = 11,
                      fig.height     = 7,
                      message        = FALSE)

# set to two decimal
options(scipen = 1, digits = 4)

In [ ]:
library(dplyr)
library(nlme)
library(purrr)
library(performance)
library(tibble)
library(here)
library(reactablefmtr)
library(reactable)
library(emmeans)
library(car)
library(ggplot2)
library(modelr)
library(knitr)

In [ ]:
# Set knit directory
setwd(here::here())
knitr::opts_knit$set(root.dir = setwd(here::here()))
getwd()

# Load functions and data

In [ ]:
# Load all joined dataset
source("./scripts/code_join_data_full_dataset.R")

# Step was done like this because I am working with a subset of the data
# source cleaned data
source("./scripts/code_clean_data_nodules.R")

In [ ]:
# Load custom made functions
source("./R/functions_models.R")
source("./R/function_plots.R")
source("./R/function_validation_plots.R")
source("./R/function_for_inference_emmeans_and_percentage_diff.R")

__Q2: How does increased nutrient and/or water availability influence seedling
water- and nutrient-use traits and the relationships with N-fixing bacteria?__

# Model fitting

## Traits

In [ ]:
# Take response variables names
response_vars_q2 <-
  set_names(c("amax", "gs", "sla"))

In [ ]:
## Create empty list
models_q2 <- list()

In [ ]:
model_q2_amax <- lme(amax ~ nfixer*treatment +init_height,
                                        random = ~1|spcode,
                                        data = data_for_models)

model_q2_amax <- list(model_q2_amax)

names(model_q2_amax) <- "amax"



In [ ]:
model_q2_gs <- lme(gs ~ nfixer*treatment +init_height,
                                        random = ~1|spcode,
                                        data = data_for_models)

model_q2_gs <- list(model_q2_gs)

names(model_q2_gs) <- "gs"


In [ ]:
model_q2_sla <- lme(sla ~ nfixer*treatment +init_height,
                                        random = ~1|spcode,
                                        data = data_for_models)

model_q2_sla <- list(model_q2_sla)

names(model_q2_sla) <- "sla"                      

In [ ]:
## PNUE
model_q2_pnue_log <- lme(log(pnue) ~ nfixer*treatment +init_height,
                                        random = ~1|spcode,
                                        data = data_for_models)

model_q2_pnue_log <- list(model_q2_pnue_log)

names(model_q2_pnue_log) <- "pnue_log"

In [ ]:
model_q2_pnue_nlme <- lme(pnue ~ nfixer*treatment + init_height,
                                        random = ~1|spcode,
                                        weights = varIdent(form = ~1|spcode*treatment),
                                        data = data_for_models)

model_q2_pnue_nlme <- list(model_q2_pnue_nlme)

names(model_q2_pnue_nlme) <- "pnue_nlme"


In [ ]:
## Narea_g_m2 log model
model_q2_n_area_log <- lme(log(narea_g_m2) ~ nfixer*treatment + init_height,
                                                random = ~1|spcode,
                                                data = data_for_models)

model_q2_n_area_log <- list(model_q2_n_area_log)

names(model_q2_n_area_log) <- "n_area_log"

In [ ]:
model_q2_n_area_nlme <- lme(narea_g_m2 ~ nfixer*treatment + init_height,
                                        random = ~1|spcode,
                                        weights = varIdent(form = ~1|spcode*treatment),
                                        data = data_for_models)

model_q2_n_area_nlme <- list(model_q2_n_area_nlme)

names(model_q2_n_area_nlme) <- "n_area_nlme"


In [ ]:
## WUE log model
model_q2_wue_log <- lme(log(wue) ~ nfixer*treatment + init_height,
                                        random = ~1|spcode,
                                        data = data_for_models)

model_q2_wue_log <- list(model_q2_wue_log)

names(model_q2_wue_log) <- "wue_log"

In [ ]:
model_q2_wue_nlme <- lme(wue ~ nfixer*treatment + init_height,
                                        random = ~1|spcode,
                                        weights = varIdent(form = ~1|spcode*treatment),
                                        data = data_for_models)

model_q2_wue_nlme <- list(model_q2_wue_nlme)

names(model_q2_wue_nlme) <- "wue_nlme"

## Nodule colonization

In [ ]:
# Delete unused variables
data_nodules_cleaned <-
    data_nodules_cleaned %>%

        # add id to rownames for keep track of the rows
        column_to_rownames("id") %>%
        dplyr::select(spcode, treatment, everything())

### Nodule weight

In [ ]:
nlme_nodule_weight <- lme(estimated_total_nodule_mass_per_plant ~ treatment + init_height,
                                    random = ~1|spcode,
                                    weights = varIdent(form = ~1|spcode*treatment),
                                    data = data_nodules_cleaned)


model_q2_nodule_weight <- list(nlme_nodule_weight)

names(model_q2_nodule_weight) <- "nodule_weight"

### Nodule count

In [ ]:
nlme_nodule_count <- lme(total_number_of_plant_nodules ~ treatment + init_height,
                                    random = ~1|spcode,
                                    weights = varIdent(form = ~1|spcode*treatment),
                                    data = data_nodules_cleaned)


model_q2_nodule_count <- list(nlme_nodule_count)

names(model_q2_nodule_count) <- "nodule_count"

In [ ]:
# Append models to model list
models_q2 <- append(model_q2_amax, models_q2)
models_q2 <- append(model_q2_gs, models_q2)
models_q2 <- append(model_q2_sla, models_q2)

models_q2 <- append(model_q2_n_area_log, models_q2)
models_q2 <- append(model_q2_n_area_nlme, models_q2)

models_q2 <- append(model_q2_pnue_log, models_q2)
models_q2 <- append(model_q2_pnue_nlme, models_q2)

models_q2 <- append(model_q2_wue_log, models_q2)
models_q2 <- append(model_q2_wue_nlme, models_q2)

models_q2 <- append(model_q2_nodule_count, models_q2)
models_q2 <- append(model_q2_nodule_weight, models_q2)

In [ ]:
names(models_q2)

# Model Assumptions

## Maximal photosynthesis

In [ ]:
validation_plots(models_q2$amax, data = data_for_models, group = "spcode")

## Stomatal Conductance

In [ ]:
validation_plots(models_q2$gs, data = data_for_models, group = "spcode")

## SLA

In [ ]:
validation_plots(models_q2$sla, data = data_for_models, group = "spcode")

## Water Use Efficiency

In [ ]:
validation_plots(models_q2$wue_log, data = data_for_models, group = "spcode")

In [ ]:
validation_plots(models_q2$wue_nlme, data = data_for_models, group = "spcode")

## PNUE

In [ ]:
validation_plots(models_q2$pnue_log, data = data_for_models, group = "spcode")

In [ ]:
validation_plots(models_q2$pnue_nlme, data = data_for_models, group = "spcode")

## Nitrogen concentration per unit of area

In [ ]:
validation_plots(models_q2$n_area_log, data = data_for_models, group = "spcode")

In [ ]:
validation_plots(models_q2$n_area_nlme, data = data_for_models, group = "spcode")

## Nodule weight

In [ ]:
validation_plots(models_q2$nodule_weight, data = data_nodules_cleaned, group = "treatment")

## Nodule count

In [ ]:
validation_plots(models_q2$nodule_count, data = data_nodules_cleaned, group = 'spcode')

# Model inference

In [ ]:
## r2 models
models_q2 %>%
    map(., r2) %>%
    unlist()

In [ ]:
## r2 models
models_q2 %>%
    map(., AIC) %>%
    unlist()

## Anova tables

In [ ]:
#map(models_q2, ~Anova(.x, type = "III", test.statistic = c("F")))

In [ ]:
map(models_q2, ~anova.lme(.x, type = "marginal"))

## Post-Hoc: Tukey's test

### Maximal photosynthesis

In [ ]:
as_tibble(emmeans(models_q2$amax,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
# Treatment effects
emmeans_table_tidy(models_q2$amax,
                        formula = "treatment|nfixer",
                        grouping_var = "nfixer")

### Water Use Efficiency

In [ ]:
as_tibble(emmeans(models_q2$wue_log,
        pairwise ~ treatment*nfixer,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable() 

In [ ]:
# Treatment effects
emmeans_table_tidy(models_q2$gs,
                        formula = "treatment",
                        )

### PNUE

In [ ]:
as_tibble(emmeans(models_q2$pnue_log,
        pairwise ~ treatment,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
# Treatment effects
emmeans_table_tidy(models_q2$pnue_log,
                        formula = "treatment",
                        )

### Nitrogen concentration per unit of area

In [ ]:
as_tibble(emmeans(models_q2$n_area_log,
        pairwise ~ treatment,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
# Treatment effects
emmeans_table_tidy(models_q2$n_area_log,
                        formula = "treatment",
                        )

In [ ]:
as_tibble(emmeans(models_q2$n_area_log,
        pairwise ~ nfixer,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
# Treatment effects
as.data.frame(emmeans::emmeans(models_q2$n_area_log,
                                specs = pairwise ~nfixer,
                                type = "response",
                                adjust = "tukey")$emmeans) %>%

        janitor::clean_names() %>%
        dplyr::select(response, everything(),
                        # Remove variables
                      -c(df, lower_cl, upper_cl, se)) %>%

        # Rename response to emmean, this is done when models is log
        dplyr::rename_all(funs(stringr::str_replace_all(., "response", "emmean"))) %>%

        # Calculate % difference between control and variable, this assume that
        # that first name is the control

        dplyr::mutate(difference = ((emmean - first(emmean))),
               perc_difference =((emmean - first(emmean) )/first(emmean))*100) %>%

        dplyr::mutate_if(is.numeric, round, 3)

### Boxplots traits

In [ ]:
# Step done for getting predictions from models for Q2
data_for_predictions <-
    data_for_models %>%

        rownames_to_column("id") %>%

        # Remove unused variables
        dplyr::select(id, spcode, treatment, nfixer, init_height)

In [ ]:
string <- c("models_q2")

data_pred_traits <-

        # Get models prediction
        gather_predictions(data_for_predictions ,

                           # Return predictions
                            models_q2$amax,
                            models_q2$sla,
                            models_q2$gs,
                            models_q2$wue_log,
                            models_q2$pnue_log,
                            models_q2$n_area_log

                            ) %>%

        pivot_wider(names_from = model, values_from = pred) %>%
            rename_all(funs(

                # rename columns
                stringr::str_to_lower(.) %>%
                stringr::str_replace(., c(string),"pred_") %>%

                # Remove dollar sing
                gsub("\\$", "", .)
                )) %>%

        # Back transform log variables
        mutate(pred_wue = exp(pred_wue_log),
                pred_n_area = exp(pred_n_area_log),
                pred_pnue =  exp(pred_pnue_log),

            ) %>%

        # Remove log predictions and init height
        dplyr::select(-c(init_height, pred_wue_log, pred_n_area_log))

In [ ]:
# Generate plot combinations

vars_q2_interaction <-

  crossing(

    # Get all numeric variables to plot (all y)
    as_tibble(t(combn(dplyr::select(data_pred_traits, where(is.numeric)) %>% names, 1))),

    # Select factor variables to plot
    x_axis_var = dplyr::select(data_pred_traits, nfixer) %>%  names,
    group_var = dplyr::select(data_pred_traits, treatment) %>%  names) %>%

    filter(V1 %in% c('pred_amax', 'pred_wue'))

In [ ]:
vars_q2_interaction %>%
      # Gererate plots
      pmap( ~ boxplot_plot_pmap(data = data_pred_traits,
                                y = !!sym(..1), x = !!sym(..2),
                                fill = !!sym(..3)))

In [ ]:
vars_q2_treatment <-

  crossing(

    # Get all numeric variables to plot (all y)
    as_tibble(t(combn(dplyr::select(data_pred_traits, where(is.numeric)) %>% names, 1))),

    # Select factor variables to plot
    x_axis_var = dplyr::select(data_pred_traits, treatment) %>%  names,
    group_var = dplyr::select(data_pred_traits, treatment) %>%  names) %>%

    filter(V1 %in% c('pred_gs', 'pred_n_area', 'pred_pnue' ))

In [ ]:
vars_q2_treatment %>%
      # Gererate plots
      pmap( ~ boxplot_plot_pmap(data = data_pred_traits,
                                y = !!sym(..1), x = !!sym(..2),
                                fill = !!sym(..3)))

### Nodule weight

In [ ]:
as_tibble(emmeans(models_q2$nodule_weight,
        pairwise ~ treatment,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
# Treatment effects
emmeans_table_tidy(models_q2$nodule_weight,
                        formula = "treatment")

### Nodule Count

In [ ]:
as_tibble(emmeans(models_q2$nodule_count,
        pairwise ~ treatment,
        adjust = "tukey"
        )$contrast) %>%
        mutate(across(2:6, round, 6)) %>%
        kable()

In [ ]:
# Treatment effects
emmeans_table_tidy(models_q2$nodule_count,
                        formula = "treatment")

### Boxplots Nodules

In [ ]:
data_for_nodule_predictions <- 
        data_nodules_cleaned %>%

        rownames_to_column("id") %>%

        # Remove unused variables
        dplyr::select(id, spcode, treatment, init_height)

In [ ]:
string <- c("models_q2")

data_pred_nodules <-

        # Get models prediction
        gather_predictions(data_for_nodule_predictions,

                           # Return predictions
                            models_q2$nodule_count,
                            models_q2$nodule_weight) %>%    

        pivot_wider(names_from = model, values_from = pred) %>%
            rename_all(funs(

                # rename columns
                stringr::str_to_lower(.) %>%
                stringr::str_replace(., c(string),"pred_") %>%

                # Remove dollar sing
                gsub("\\$", "", .)
                )) %>% 
        # Remove log predictions and init height
        dplyr::select(-c(init_height))

In [ ]:
# Generate plot combinations

vars_q2_nodules <-

  crossing(

    # Get all numeric variables to plot (all y)
    as_tibble(t(combn(dplyr::select(data_pred_nodules, where(is.numeric)) %>% names, 1))),

    # Select factor variables to plot
    x_axis_var = dplyr::select(data_pred_nodules, treatment) %>%  names,
    group_var = dplyr::select(data_pred_nodules, treatment) %>%  names) 

In [ ]:
vars_q2_nodules %>%
      # Gererate plots
      pmap( ~ boxplot_plot_pmap(data = data_pred_nodules,
                                y = !!sym(..1), x = !!sym(..2),
                                fill = !!sym(..3)))